In [4]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import pyarrow.feather as feather

In [5]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [ ]:
sns.barplot()

In [9]:
cause_effect_table= pd.pivot_table(export_df, index=['cause'], columns='effect',aggfunc='count',fill_value=0,values="emoji", margins=True, margins_name= 'Total')

KeyError: 'cause'